Burrows-Wheeler-Transform

In [9]:
def bwt(s):
    assert"\003" not in s, "Input string cannot contain ETX characters"
    s = s + "\003"
    table = sorted(s[i:] + s[:i] for i in range(len(s))) 
    print("table done")
    last_column = [row[-1:] for row in table]
    return "".join(last_column)

Run Length Encoding

In [7]:
from collections import OrderedDict

def runLengthEncoding(input): 
    dict=OrderedDict.fromkeys(input, 0) 
    for ch in input: 
        dict[ch] += 1
    output = '' 
    for key,value in dict.items(): 
         output = output + key + str(value) 
    return output 

Lempel-Ziv Algorithm

In [ ]:
class Lz77:
    def __init__(self, sliding_window_size, search_buffer_size):
        self.sws = sliding_window_size
        self.sbs = search_buffer_size
        self.encoded_text = ""
        self.codes = []
    
    def compressor(self,data):
        encoded_text  = ""
        position = 0 
        offset = 0
        max_match_length = 0
        while(position<len(data)):
            #print("a")
            offset,max_match_length,codeword =  self.match(position,data)
            self.codes.append([offset,max_match_length,codeword])
            position += max_match_length+1
            #print("c", max_match_length)
            encoded_text = encoded_text+str(offset)+" "+ str(max_match_length)+" "+codeword+" "
        self.encoded_text = encoded_text
        return encoded_text

    def match(self, position, data):
        sb = self.sbs
        i = 1
        key = data[position]
        max_match_length = 0
        offset = 0
        while((position-i)>=0 and i!=sb+1):
            #print("b")
            length = 0
            if(data[position-i]==key):
                length+=1
                j = 1
                while(position-i+j!= len(data) and position+j!=len(data)):
                    if(data[position-i+j]== data[position+j]):
                        length+=1
                        j+=1
                    else:
                        break
                if(max_match_length<length):
                    max_match_length = length
                    offset = i
            i +=1
            if(position+max_match_length==len(data)):
                return offset,max_match_length,""
        return offset,max_match_length,data[position+max_match_length]

    def decompressor(self):
        decoded_text = ""
        for code in (self.codes):
            pointer_position = len(decoded_text)
            offset = code[0]
            length = code[1]
            code_word = code[2]
            for j in range(length):
                decoded_text += decoded_text[pointer_position-offset+j] 
            decoded_text+=code_word

        return decoded_text

Huffman Encoding

In [8]:
import heapq
import pandas as pd 
import numpy as np 
from collections import Counter
import re
from functools import total_ordering

@total_ordering
class Node:
    def __init__(self, freq,char):
        self.left = None
        self.right= None
        self.fre = freq
        self.key = char
    
    # defining comparators less_than and equals
    def __lt__(self, other):
        return self.fre < other.fre

    def __eq__(self,other):
        if(other == None):
            return False
        if(not isinstance(other , Node)):
            return False
        return self.fre == other.fre

class Tree:
    def __init__(self,listfreq):
        self.heap = []
        self.codes = {}
        self.reverse_mapping = {}
        for key in listfreq:
            node = Node(listfreq[key], key)
            heapq.heappush(self.heap,node)

        while(len(self.heap)>1):
            t1 = heapq.heappop(self.heap)
            t2 = heapq.heappop(self.heap)
            count = t1.fre + t2.fre
            D = Node(count,None)
            D.left = t1
            D.right = t2
            heapq.heappush(self.heap, D)
	
    
    def huffmancode(self, root, current_code):
        if (root == None) :
            return

        if(root.key != None):
            self.codes[root.key] = current_code
            self.reverse_mapping[current_code] = root.key
            return
        
        self.huffmancode(root.left,current_code+"0")
        self.huffmancode(root.right, current_code+"1")

    def compress(self, text):
        encoded_text =  "".join(self.codes[let] for let in text)
        extra_padding = 8 - len(encoded_text)%8
        for i in range(extra_padding):
            encoded_text += "0"
        
        padded_count = "{0:08b}".format(extra_padding)

        padded_encoded_text = padded_count + encoded_text

        if(len(padded_encoded_text)%8 !=0):
            print(" NOt padded properly")
            exit(0)

        b = bytearray()

        for i in range(0, len(padded_encoded_text), 8):
            byte = padded_encoded_text[i:i+8]
            b.append(int(byte,2))
        
        output_path = "compressed.bin"
        output = open(output_path, 'wb')

        output.write(bytes(b))

        print("Compressed Successfully")
        return output_path
